# 1. Structure
In this chapter, we will start with the first step of molecular dynamics simulation, structure preparation.

## 1.1. Polyethylene molecule
Create a polyethylene molecule in Avogadro 2 and save it as "PE.xyz".
- Monomer of polyethylene is C2H4.
- For simplicity, degree of polymerization is 10, i.e., C20H42.

![](https://media.geeksforgeeks.org/wp-content/uploads/20240123131906/Polyethylene.jpg)

## 1.2. Pack molecules

Define a function to calculate cubic simulation box size by defining density and number of molecules.
- In this tutorial, we limit our size to 30 molecules.

In [1]:
def get_box_size(density, num):
    # Molecular weight
    C = 12.0107
    H = 1.00794
    PE = 20*C + 42*H

    # Unit conversion
    amu_g = 1.66054e-24
    Å_cm = 1e-8
    amuÅ3_gcm3 = amu_g / (Å_cm**3)
    density_amuÅ3 = density / amuÅ3_gcm3
    volume = num*PE / density_amuÅ3

    # Assume cubic box
    box_size = volume**(1/3)
    return box_size

Print out the box size
- Initial density is 0.3 g/cm^3 to avoid undesirable overlap

In [2]:
density = 0.3  # g/cm^3
num = 30
box_size = get_box_size(density, num)
x = box_size
y = x
z = x
print(f'Box size: x = {x:.2f} Å, y = {y:.2f} Å, z = {z:.2f} Å.')

Box size: x = 36.07 Å, y = 36.07 Å, z = 36.07 Å.


## 1.3. Packmol
As the name suggests, it is written for packing molecules. Visit [Packmol](https://m3g.github.io/packmol/index.html) for documentation.

Define variables for later use

In [1]:
s = 1.25            # Vacuum space in packing Kapton
output = 'data'     # Output file name
input = 'PE'        # Input file name

Write input file for Packmol

In [4]:
with open('in.packmol', 'w') as f:
    f.write(f'tolerance {s*2:.2f}\n')
    f.write('filetype xyz\n')
    f.write(f'output {output}.xyz\n')
    
    f.write(f'structure {input}.xyz\n')
    f.write(f'  number {num}\n')
    f.write(f'  inside box {s:.2f} {s:.2f} {s:.2f} {x-s:.2f} {y-s:.2f} {z-s:.2f}\n')     # 1.25 Å vaccum
    f.write('end structure\n')

Write input file for julia to run Packmol

In [5]:
from pathlib import Path
input_file = Path.cwd()/'in.packmol'

with open('in.julia', 'w') as f:
    f.write('import Pkg; Pkg.activate("Packmol", shared=true) # Activate Packmol environment\n')
    f.write('using Packmol\n')
    f.write(f'input_file = raw"{input_file}"\n')
    f.write('run_packmol(input_file)\n')

Run Packmol in julia

In [6]:
!julia in.julia


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.3 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: xyz
  Seed for random number generator:      1234567
  Output file: data.xyz
  Reading coordinate file: PE.xyz
  Number of independent structures:            1
  The structures are: 
  Structure            1 :XYZ file generated by Avogadro.(          62  atoms)
  Maximum number of GENCAN loops for all molecule packing:          200
  Total number of restrictions:            1
  Distance tolerance:    2.5000000000000000     
  Number of 

  Activating new project at `C:\Users\fitbe\.julia\environments\Packmol`


## 1.4. Lammps Data file

Export LAMMPS data file using OVITO Python

In [7]:
import ovito.io as io
pipeline = io.import_file(location=f'{output}.xyz', input_format='xyz', columns=['Particle Type', 'Position.X', 'Position.Y', 'Position.Z'])
io.export_file(data=pipeline, file=f'{output}.lmp', format='lammps/data', atom_style='charge')

Change simulation box in data file

In [ ]:
with open(f'{output}.lmp', 'r') as f:
    lines = f.readlines()

with open(f'{output}.lmp', 'w') as f:
    for line in lines:
        if 'xlo xhi' in line:
            line = f'0 {x:.2f} xlo xhi\n'
        elif 'ylo yhi' in line:
            line = f'0 {y:.2f} ylo yhi\n'
        elif 'zlo zhi' in line:
            line = f'0 {z:.2f} zlo zhi\n'
        f.write(line)

Check if data file is correct
- Visualize by openning it in OVITO